In [10]:
import json
import numpy as np
import h5py
import matplotlib.pyplot as plt
from tqdm import tqdm_notebook
from torchvision import models
from torchvision import transforms
import torch
from torch.utils.data import Dataset, DataLoader
from sklearn.cluster import MiniBatchKMeans
import os

In [11]:
with h5py.File("data02/london_lite_gt.h5","r") as f:
    fovs = f["fov"][:]
    sim = f["sim"][:].astype(np.uint8)

In [12]:
def bag_of_words(centroids, img_descriptors):
    n_centroids = centroids.shape[0]  # number of centroids found with the KMeans clustering
    n_descriptors = img_descriptors.shape[0]  # number of descriptors extracted from the image

    bow_vector = np.zeros(n_centroids)

    for i in range(n_descriptors):
        idx = np.argmin(np.linalg.norm(centroids - img_descriptors[i], axis=1), axis=0)
        bow_vector[idx] += 1
    return bow_vector

In [13]:
class CustomDataset(Dataset):
    def __init__(self, root_dir, json_path, transform=None):
        self.transform=transform
        self.root_dir = root_dir
        with open(json_path,"r") as f:
            m_idx = json.load(f)
            self.m_imgs = np.array(m_idx["im_paths"])

    def __len__(self):
        return len(self.m_imgs)

    def __getitem__(self, idx):
        img = plt.imread(os.path.join(self.root_dir, self.m_imgs[idx]))

        if self.transform:
            img = self.transform(img)
        return img

In [24]:
# def create_deep_descriptors():
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Loaded device: {device}")

model = models.resnet50(weights=models.ResNet50_Weights.IMAGENET1K_V2)
model = torch.nn.Sequential(*(list(model.children())[:-1]))
model = model.to(device)

Loaded device: cuda


In [25]:
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Resize((256, 256), antialias=False)
])

batch_size = 64

database = CustomDataset(root_dir="data02", json_path="data02/database/database_lite.json", transform=transform)
loader = DataLoader(database, batch_size=batch_size)

kmeans = MiniBatchKMeans(n_clusters=32, random_state=0, batch_size=batch_size, n_init=3)

# Find centroids
print("Finding centroids...")
for img in tqdm_notebook(loader):
    img = img.to(device)
    with torch.no_grad():
        output = model(img)

        kmeans.partial_fit(output.cpu().numpy()[:, :, 0, 0])

centroids = kmeans.cluster_centers_

# Create BoVW vectors
print("Finding centroids...")
bow_map_images = None
for img in tqdm_notebook(loader):
    img = img.to(device)
    with torch.no_grad():
        output = model(img)
        for i_batch in range(output.shape[0]):
            bow = bag_of_words(centroids, output.cpu().numpy()[i_batch, :, 0, 0])
            if bow_map_images is None:
                bow_map_images = bow
            else:
                bow_map_images = np.vstack( (bow_map_images, bow))

Finding centroids...


C:\Users\koend\AppData\Local\Temp\ipykernel_25536\2551632580.py:15: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  for img in tqdm_notebook(loader):


  0%|          | 0/16 [00:00<?, ?it/s]

KeyboardInterrupt: 

In [ ]:
def split_image(img, n):
    print("split")